# Bias-Removing Autoencoder for Reject Inference
### Group 1 - Seminar Information Systems (WiSe 2021/22)
#### Klemens Lehmann (), Jan Papmeier (), Lukas Voege (615033)

### Table of Contents

1. Introduction
1. Problem Setting
    1. _Credit Scroring_
    1. _Acceptance Loop and Sampling Bias_
1. Current Methods in Reject Inference
    1. _Basic Concepts of Reject Inference_
    1. _List of reject inference Ideas/Methods_
    1. _*TEST OF OTHER METHODS??*_
1. Reject Inference with Autoencoders
    1. _What is an Autoencoder_
    1. _How could an Autoencoder help with sampling selection bias? - Loss Function_
1. Testing and Results
1. Conclusion

## 1. Introduction


## 2. Problem Setting
#### 2.1 Credit Scoring
Credit scoring models evaluate the default risk (probability of not paying back) of any applicant applying for a loan. Several Methods have been developed since Durant first publication in 1941. For an overview see Lessmann et al. (2015). Credit Scoring models are trained on accepted cases only. This creates a sampling bias, since rejected cases are left out and therefore are not representive for the whole population of possible borrowers. Historically, Reject Inference methods have been used to tackle the problem. In Reject Inference we build a model for the accepted cases and extrapolate it for the rejected ones using several methods, which we will discuss in a seperate section. In this notebook we try to add a special type of neural network, namely an Autoencoder, to our Reject Inference methods to achieve a reduction of the sampling bias.

#### 2.2 Acceptance Loop and Sampling Bias

## 3. Current Methods in Reject Inference
#### 3.1 Basic Concepts of Reject Inference

#### 3.2 List of reject inference Ideas/Methods

#### 3.3 TEST OF OTHER METHODS??

#### 3.4 Theoretical Background of Assumptions on Nature of Missing Data points

## 4. Reject Inference with Autoencoders
#### 4.1 What is an Autoencoder

#### 4.2 How could an Autoencoder help with sampling selection bias? - Loss Function

## 5. Testing and Results

## 6. Conclusion